# Classification

In [1]:
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib
import statsmodels.api as sm
import xlsxwriter
import math
from statsmodels.regression.rolling import RollingOLS
import holoviews as hv

#data
data = pd.read_csv('aussie_equities_scoring_python_15072024.csv', index_col=0, encoding='latin-1')
data1 = data.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data2 = data1.copy()
data2.columns = [data1.columns, data.iloc[0]]
data1.columns = data.iloc[0]
data_cleaned = data1.iloc[1:]

#tracking error
excess_return = data_cleaned.iloc[:,5:].sub(data_cleaned.iloc[:,:1]['Market'], axis=0)
tracking_errors = excess_return.std() * math.sqrt(12)
tracking_errors.index = data2.iloc[:,5:].columns

#beta estimation
output = {}
output2 = {}
for i in data_cleaned.iloc[:,5:].columns:
    y = data_cleaned[i]
    x = data_cleaned.iloc[:,:5]
    if y.count() < 24:
            output[i] = np.nan
            output2[i] = np.nan
    else: 
        model = sm.OLS(y, sm.add_constant(x), missing='drop').fit()
        output[i] = model.params
        output2[i] = model.tvalues

coefficients = pd.DataFrame(output)
tstats = pd.DataFrame(output2)
coefficients.columns = data2.iloc[:,5:].columns
tstats.columns = data2.iloc[:,5:].columns

#classification
classifications = []
for i in tstats.columns:
    ranks = tstats[i].sort_values(axis=0, ascending = False)
    ranks_cleaned = ranks.drop(['Market','const'],axis=0)

    if tracking_errors.loc[i] < 0.01:
        classifications.append('Beta')
        
    elif ranks_cleaned.iloc[0] > 2.581:
        if not (ranks_cleaned.index[-1] == 'Value' and ranks_cleaned.loc['Value'] < -2.581 and (abs(ranks_cleaned.loc['Value']) > ranks_cleaned.iloc[0])):
            classifications.append(ranks_cleaned.index[0])
        else:
            classifications.append('Growth')

    elif (ranks_cleaned.loc['Value'] < -2.581 and (abs(ranks_cleaned.loc['Value']) > ranks_cleaned.iloc[0])):
        classifications.append('Growth')

    elif tracking_errors.loc[i] < 0.02:
        classifications.append('Beta')

    else:
        classifications.append('Alpha')
    

final_df = tstats.T
final_df['Classification'] = classifications
final_df['Tracking Error'] = tracking_errors

In [3]:
final_df

,,const,Market,Value,Size,Momentum,Quality,Classification,Tracking Error
,apir_ticker,,,,,,,,
Tyndall Australian Share Income,TYN0038AU,-1.118164,38.707775,5.233580,2.216515,-0.898514,-2.306161,Value,0.044625
Bennelong Concentrated Australian Eq,BFL0002AU,0.174275,22.962758,-2.424733,4.213851,2.701743,2.156893,Size,0.087037
Bennelong Australian Equities,BFL0001AU,-0.317738,29.536625,-3.691536,3.456338,2.440428,1.794759,Growth,0.069432
Solaris Core Australian Equity Inst,WHT0009AU,0.336730,89.773578,-4.578660,-1.734467,0.476195,-3.394085,Growth,0.021468
Ellerston Australian Share Fund,ECL0005AU,-0.097033,21.260345,0.583486,4.324945,-1.966181,0.130728,Size,0.072799
...,...,...,...,...,...,...,...,...,...
IML Concntr AUS Shr Fd (Quoted Mngd Fd),IMLC,NaN,NaN,NaN,NaN,NaN,NaN,Alpha,0.031693
Dimensional AustralianCoreEquityTrustETF,DACE,-1.748532,107.458020,9.020765,16.462789,-2.191389,-2.963473,Size,0.031162
BetasharesWthBdrAUS200Grd30-40%LVRCpxETF,G200,NaN,NaN,NaN,NaN,NaN,NaN,Alpha,NaN


Find:

In [ ]:
final_df.xs('DNR Capital Aus Eq High Conviction R',axis=0,level=0)

In [2]:
exog = sm.add_constant(data_cleaned.iloc[:,:5], prepend=False)
mod = RollingOLS(data_cleaned['PIM0028AU'], exog, window=36, missing='drop').fit()
mod.params.index = pd.to_datetime(mod.params.index)
plot = mod.params.dropna().iloc[10:,1:-1].hvplot(rot=90, title='DNR', width=800, height=500)
hline = hv.HLine(0).opts(color='gray',line_width = 0.75)
combo = plot * hline
combo

c:\Users\JohnBilsel\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
C:\Users\JohnBilsel\AppData\Local\Temp\ipykernel_10572\163112438.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  mod.params.index = pd.to_datetime(mod.params.index)


:Overlay
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [name]   (value)
   .HLine.I     :HLine   [x,y]

In [20]:
final_df.to_csv('new_scoring.csv')